In [ ]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [ ]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

In [ ]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features # Convolution

        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        ) # FC layer

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) # Convolution
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1)
        x = self.classifier(x) # FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') # mode='fan_out : backwards pass
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [ ]:
# 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

# model = make_layer(cfg['A'], batch_norm=False)

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v

    return nn.Sequential(*layers)

In [ ]:
# conv2d = nn.Conv2d(3, 64, kernel_size=3, padding=1), #1
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv3d(64, 128, kernel_size=3, padding=1) #2
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(128, 256, kernel_size=3, padding=1) #3
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1) #4
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(256, 512, kernel_size=3, padding=1) #5
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #6
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #7
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

# conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #8
# nn.ReLU(inplace=True)

# nn.MaxPool2d(kernel_size=2, stride=2)

In [ ]:
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #8 + 3 =11 == vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
    'custom' : [64,64,64,'M',128,128,128,'M',256,256,256,'M']
}

In [ ]:
conv = make_layers(cfg['A'])

In [ ]:
conv

In [ ]:
conv = make_layers(cfg['custom'], batch_norm=True)

In [ ]:
CNN = VGG(make_layers(cfg['custom']), num_classes=10, init_weights=True)

In [ ]:
CNN